# Joint Bert Model for slot and intent classification

### Imports

In [93]:
import json
import os
import re
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from transformers import TFBertModel
from tensorflow.keras.layers import Dropout, Dense, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy, CategoricalAccuracy, BinaryAccuracy

model_name = "bert-base-uncased"

# connect MLFlow
import mlflow
mlflow.login()

# set the experiment id
mlflow.set_experiment(experiment_id="1400101170489055")

mlflow.tensorflow.autolog()

2024/10/05 17:11:22 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://community.cloud.databricks.com.


### Load Dataset

In [94]:
inputs = []
intentOutputs = []
slotOutputs = []

with open("../processing/JERTmate_final_data.json", "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

    inputs = data["inputs"]
    intentOutputs = data["intentOutputs"]
    slotOutputs = data["slotOutputs"]

### Split Data - Train 80% | Validation 10% | Test 10%

In [95]:
def split_arrays(inputs, intentOutputs, slotOutputs, train_ratio, val_ratio, test_ratio):
    assert len(inputs) == len(intentOutputs) == len(slotOutputs), "All arrays must have the same length"
    
    n_total = len(inputs)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)
    
    # split inputs
    inputs_train, inputs_val, inputs_test = inputs[:n_train], inputs[n_train:n_train + n_val], inputs[n_train + n_val:]

    # split intents
    intentOutputs_train, intentOutputs_val, intentOutputs_test = intentOutputs[:n_train], intentOutputs[n_train:n_train + n_val], intentOutputs[n_train + n_val:]

    # split slots
    slot_type_map_train, slot_type_map_val, slot_type_map_test = [x[:50] for x in slotOutputs[:n_train]], [x[:50] for x in slotOutputs[n_train:n_train + n_val]], [x[:50] for x in slotOutputs[n_train + n_val:]]
    slot_intent_map_train, slot_intent_map_val, slot_intent_map_test = [x[50:100] for x in slotOutputs[:n_train]], [x[50:100] for x in slotOutputs[n_train:n_train + n_val]], [x[50:100] for x in slotOutputs[n_train + n_val:]]
    slot_action_map_train, slot_action_map_val, slot_action_map_test = [x[100:150] for x in slotOutputs[:n_train]], [x[100:150] for x in slotOutputs[n_train:n_train + n_val]], [x[100:150] for x in slotOutputs[n_train + n_val:]]
    slot_pointers_map_train, slot_pointers_map_val, slot_pointers_map_test = [x[150:300] for x in slotOutputs[:n_train]], [x[150:300] for x in slotOutputs[n_train:n_train + n_val]], [x[150:300] for x in slotOutputs[n_train + n_val:]]
    phantom_target_map_train, phantom_target_map_val, phantom_target_map_test = [x[300:305] for x in slotOutputs[:n_train]], [x[300:305] for x in slotOutputs[n_train:n_train + n_val]], [x[300:305] for x in slotOutputs[n_train + n_val:]]
    phantom_intent_map_train, phantom_intent_map_val, phantom_intent_map_test = [x[305:310] for x in slotOutputs[:n_train]], [x[305:310] for x in slotOutputs[n_train:n_train + n_val]], [x[305:310] for x in slotOutputs[n_train + n_val:]]
    phantom_action_map_train, phantom_action_map_val, phantom_action_map_test = [x[310:315] for x in slotOutputs[:n_train]], [x[310:315] for x in slotOutputs[n_train:n_train + n_val]], [x[310:315] for x in slotOutputs[n_train + n_val:]]
    phantom_pointers_map_train, phantom_pointers_map_val, phantom_pointers_map_test = [x[315:] for x in slotOutputs[:n_train]], [x[315:] for x in slotOutputs[n_train:n_train + n_val]], [x[315:] for x in slotOutputs[n_train + n_val:]]

    
    return (tf.constant(inputs_train), tf.constant(inputs_val), tf.constant(inputs_test)), (tf.constant(intentOutputs_train), tf.constant(intentOutputs_val), tf.constant(intentOutputs_test)), (tf.constant(slot_type_map_train), tf.constant(slot_type_map_val), tf.constant(slot_type_map_test)), (tf.constant(slot_intent_map_train), tf.constant(slot_intent_map_val), tf.constant(slot_intent_map_test)), (tf.constant(slot_action_map_train), tf.constant(slot_action_map_val), tf.constant(slot_action_map_test)), (tf.constant(slot_pointers_map_train), tf.constant(slot_pointers_map_val), tf.constant(slot_pointers_map_test)), (tf.constant(phantom_target_map_train), tf.constant(phantom_target_map_val), tf.constant(phantom_target_map_test)), (tf.constant(phantom_intent_map_train), tf.constant(phantom_intent_map_val), tf.constant(phantom_intent_map_test)), (tf.constant(phantom_action_map_train), tf.constant(phantom_action_map_val), tf.constant(phantom_action_map_test)), (tf.constant(phantom_pointers_map_train), tf.constant(phantom_pointers_map_val), tf.constant(phantom_pointers_map_test))


train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

print(len(inputs))

(inputs_train, inputs_val, inputs_test), (intentOutputs_train, intentOutputs_val, intentOutputs_test), (slot_type_map_train, slot_type_map_val, slot_type_map_test), (slot_intent_map_train, slot_intent_map_val, slot_intent_map_test), (slot_action_map_train, slot_action_map_val, slot_action_map_test), (slot_pointers_map_train, slot_pointers_map_val, slot_pointers_map_test), (phantom_target_map_train, phantom_target_map_val, phantom_target_map_test), (phantom_intent_map_train, phantom_intent_map_val, phantom_intent_map_test), (phantom_action_map_train, phantom_action_map_val, phantom_action_map_test), (phantom_pointers_map_train, phantom_pointers_map_val, phantom_pointers_map_test) = split_arrays(inputs, intentOutputs, slotOutputs, train_ratio, val_ratio, test_ratio)

print(intentOutputs_train.shape)

34535
(10479, 330)
(20721, 38)


### Define Model

In [96]:
class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_vector_length=None, total_slot_number=None, total_phantom_slot_number=None, slot_types=None, slot_intents=None, pointer_possibilities=None, model_name=model_name, dropout_prob=0.08):
        super().__init__(name="joint_intent_slot")
        #   ** GENERAL LAYERS **
        self.bert = TFBertModel.from_pretrained(model_name) # BERT model
        self.dropout = Dropout(dropout_prob) # basic dropout layer
        self.flatten = Flatten() # flatten layer



        #   ** SLOT LAYERS **
        # LHS compressor
        #?self.conv_layer = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 768), strides=(1, 1),padding='valid',activation='relu')
        self.last_hidden_sequence_compressor = Dense(450, activation="relu", name="last_hidden_sequence_compressor")

        # processing layers
        #?self.slot_processor = Dense(((150) + (intent_vector_length * 2) + (slot_vector_length * 2)), activation="relu", name="slot_processor")

        # slot output layers
        self.slot_type_dense = Dense(total_slot_number * slot_types, activation='softmax', name="slot_type_output")
        self.slot_type_reshape = Reshape((total_slot_number, slot_types))
        
        self.slot_intent_dense = Dense(total_slot_number * slot_intents, activation='softmax', name="slot_intent_output")
        self.slot_intent_reshape = Reshape((total_slot_number, slot_intents))
        
        self.slot_action_output = Dense(total_slot_number, activation='softmax', name="slot_action_output")
        
        self.slot_pointers_dense = Dense(total_slot_number * pointer_possibilities * 3, activation='softmax', name="slot_pointers_output")
        self.slot_pointers_reshape = Reshape((total_slot_number * 3, pointer_possibilities))

        # Phantom slot output layers
        self.phantom_slot_target_dense = Dense(total_phantom_slot_number * pointer_possibilities, activation='softmax', name="phantom_slot_target_output")
        self.phantom_slot_target_reshape = Reshape((total_phantom_slot_number, pointer_possibilities))
        
        self.phantom_slot_intent_dense = Dense(total_phantom_slot_number * slot_intents, activation='softmax', name="phantom_slot_intent_output")
        self.phantom_slot_intent_reshape = Reshape((total_phantom_slot_number, slot_intents))
        
        self.phantom_slot_action_output = Dense(total_phantom_slot_number, activation='softmax', name="phantom_slot_action_output")
        
        self.phantom_slot_pointers_dense = Dense(total_phantom_slot_number * pointer_possibilities * 3, activation='softmax', name="phantom_slot_pointers_output")
        self.phantom_slot_pointers_reshape = Reshape((total_phantom_slot_number * 3, pointer_possibilities))



        #  ** INTENT LAYERS **
        # processing layers
        #?self.intent_processor = Dense(((150 * 1) + (intent_vector_length * 2)), activation="relu", name="intent_processor")

        # output layer
        self.intent_output = Dense(intent_vector_length, activation='softmax', name="intent_output")

    def __call__(self, inputs, **kwargs):
        # run BERT
        trained_bert = self.bert(inputs[:, :150], **kwargs)
        pooled_output = trained_bert.pooler_output
        sequence_output = trained_bert.last_hidden_state

        #   ** SLOT CLASSIFICATION **
        # reduce dimensionality of sequence output - Dense(150)
        flattened_sequence_output = self.flatten(sequence_output)
        LHSC_output = self.last_hidden_sequence_compressor(flattened_sequence_output)

        # slot processor
        #slot_processor_input = self.dropout(tf.concat([LHSC_output, tf.cast(inputs[:, 150:], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        #slot_processor_output = self.slot_processor(slot_processor_input)

        # slot output
        slot_output_input = self.dropout(tf.concat([LHSC_output, tf.cast(inputs[:, 150:], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        
        slot_type_output = self.slot_type_dense(slot_output_input)
        slot_type_output = self.slot_type_reshape(slot_type_output)
        
        slot_intent_output = self.slot_intent_dense(slot_output_input)
        slot_intent_output = self.slot_intent_reshape(slot_intent_output)
        
        slot_action_output = self.slot_action_output(slot_output_input)
        
        slot_pointers_output = self.slot_pointers_dense(slot_output_input)
        slot_pointers_output = self.slot_pointers_reshape(slot_pointers_output)

        # Phantom slot outputs
        phantom_target_output = self.phantom_slot_target_dense(slot_output_input)
        phantom_target_output = self.phantom_slot_target_reshape(phantom_target_output)
        
        phantom_intent_output = self.phantom_slot_intent_dense(slot_output_input)
        phantom_intent_output = self.phantom_slot_intent_reshape(phantom_intent_output)
        
        phantom_action_output = self.phantom_slot_action_output(slot_output_input)
        
        phantom_pointers_output = self.phantom_slot_pointers_dense(slot_output_input)
        phantom_pointers_output = self.phantom_slot_pointers_reshape(phantom_pointers_output)



        #   ** INTENT CLASSIFICATION **
        # intent processor
        #intent_processor_input = self.dropout(tf.concat([pooled_output, tf.cast(inputs[:, 150:150 + 114], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        #intent_processor_output = self.intent_processor(intent_processor_input)

        # intent output
        intent_output_input = self.dropout(tf.concat([pooled_output, tf.cast(inputs[:, 150:150 + 114], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        intent_output = self.intent_output(intent_output_input)

        return intent_output, slot_type_output, slot_intent_output, slot_action_output, slot_pointers_output, phantom_target_output, phantom_intent_output, phantom_action_output, phantom_pointers_output

    def get_config(self):
        config = super(JointIntentAndSlotFillingModel, self).get_config()
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)
    
joint_model = JointIntentAndSlotFillingModel(intent_vector_length=38, total_slot_number=50, total_phantom_slot_number=5, slot_types=15, slot_intents=4, pointer_possibilities=18)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

### Train and Compile

In [97]:
# optimizer
opt = Adam(learning_rate=5e-5, epsilon=1e-08)

# learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 7:
        return lr
    else:
        return lr * 0.97
learning_rate_callback = LearningRateScheduler(scheduler)

# Model checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath="model_epoch_{epoch:02d}.keras",  # Save the model with the epoch number in the filename
    save_freq='epoch',
    verbose=1 
)

# loss functions
losses = [
    CategoricalCrossentropy(name="intent_loss"), 
    SparseCategoricalCrossentropy(from_logits=True, name="slot_type_loss"),
    SparseCategoricalCrossentropy(from_logits=True, name="slot_intent_loss"),
    BinaryCrossentropy(name="slot_actionable_loss"), 
    SparseCategoricalCrossentropy(from_logits=True, name="slot_pointer_loss"),
    SparseCategoricalCrossentropy(from_logits=True, name="phantom_slot_target_loss"),
    SparseCategoricalCrossentropy(from_logits=True, name="phantom_slot_intent_loss"),
    BinaryCrossentropy(name="phantom_slot_actionable_loss"), 
    SparseCategoricalCrossentropy(from_logits=True, name="phantom_slot_pointer_loss"),
]
# metrics
metrics = [
    CategoricalAccuracy(name="intent_accuracy"), 
    SparseCategoricalAccuracy(name="slot_type_accuracy"),
    SparseCategoricalAccuracy(name="slot_intent_accuracy"),
    BinaryAccuracy(name="slot_actionable_accuracy"), 
    SparseCategoricalAccuracy(name="slot_pointer_accuracy"),
    SparseCategoricalAccuracy(name="phantom_slot_target_accuracy"),
    SparseCategoricalAccuracy(name="phantom_slot_intent_accuracy"),
    BinaryAccuracy(name="phantom_slot_actionable_accuracy"), 
    SparseCategoricalAccuracy(name="phantom_slot_pointer_accuracy"),
]

# compile model
joint_model.compile(optimizer=opt, loss=losses, metrics=metrics)

history = joint_model.fit(
    x=inputs_train, 
    y=(intentOutputs_train, slot_type_map_train, slot_intent_map_train, slot_action_map_train, slot_pointers_map_train, phantom_target_map_train, phantom_intent_map_train, phantom_action_map_train, phantom_pointers_map_train), 
    validation_data=(inputs_val, (intentOutputs_val, slot_type_map_val, slot_intent_map_val, slot_action_map_val, slot_pointers_map_val, phantom_target_map_val, phantom_intent_map_val, phantom_action_map_val, phantom_pointers_map_val)), 
    epochs=20, 
    batch_size=24,
    shuffle=True, 
    callbacks=[learning_rate_callback, checkpoint_callback],
)

2024/10/05 17:11:34 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ec1a00584e494a67a6bb346e02462261', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow
2024/10/05 17:11:34 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: 'JointIntentAndSlotFillingModel' object has no attribute 'input_shape'
2024/10/05 17:11:35 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: If 'features' is a TensorFlow Tensor, then 'targets' must also be a TensorFlow Tensor. Found: <class 'tuple'>.


Epoch 1/20
  1/864 ━━━━━━━━━━━━━━━━━━━━ 4:50:35 20s/step - intent_accuracy: 0.0417 - loss: 19.7594 - phantom_slot_actionable_accuracy: 0.6667 - phantom_slot_intent_accuracy: 0.1667 - phantom_slot_pointer_accuracy: 0.0583 - phantom_slot_target_accuracy: 0.0250 - slot_actionable_accuracy: 0.0000e+00 - slot_intent_accuracy: 0.2308 - slot_pointer_accuracy: 0.0528 - slot_type_accuracy: 0.0658

2024/10/05 17:12:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run dapper-gull-583 at: https://community.cloud.databricks.com/ml/experiments/1400101170489055/runs/ec1a00584e494a67a6bb346e02462261.
2024/10/05 17:12:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://community.cloud.databricks.com/ml/experiments/1400101170489055.


KeyboardInterrupt: 

### Evaluate Model

keep in mind -> test slot accuracy will be higher than reality.
        Because 90% of the data points are 0, it can just guess 0 and be right 85% of the time

In [ ]:
test_loss, test_slot_acc, test_intent_acc = joint_model.evaluate(x=inputs_test, y=(intentOutputs_test, slot_type_map_test, slot_intent_map_test, slot_action_map_test, slot_pointers_map_test, phantom_target_map_test, phantom_intent_map_test, phantom_action_map_test, phantom_pointers_map_test), batch_size=24)

print(f"Test Slot Accuracy: {test_slot_acc}")
print(f"Test Intent Accuracy: {test_intent_acc}")

### Inference

In [20]:
def nlu(text, tokenizer, model, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = model(inputs)
    slot_logits, intent_logits = outputs

    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, :]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]

    info = {"intent": intent_names[intent_id], "slots": {}}

    out_dict = {}
    # get all slot names and add to out_dict as keys
    predicted_slots = set([re.sub(r'^[BI]-', '', slot_names[s]) for s in slot_ids if s != 0])
    for ps in predicted_slots:
      out_dict[ps] = []

    # check if the text starts with a small letter
    if text[0].islower():
      tokens = tokenizer.tokenize(text, add_special_tokens=True)
    else:
      tokens = tokenizer.tokenize(text)

    # process sequence output for slots
    for token, slot_id in zip(tokens, slot_ids):
      # add all to out_dict
      slot_name = slot_names[slot_id]

      if slot_name == "[PAD]":
        continue

      # collect tokens
      collected_tokens = [token]
      idx = tokens.index(token)

      # see if it starts with ##
      # then it belongs to the previous token
      if token.startswith("##"):

        # check if the token already exists or not
        if tokens[idx - 1] not in out_dict[slot_name]:
          collected_tokens.insert(0, tokens[idx - 1])

      # add collected tokens to slots
      out_dict[slot_name].extend(collected_tokens)

    # process out_dict
    for slot_name in out_dict:
      tokens = out_dict[slot_name]
      slot_value = tokenizer.convert_tokens_to_string(tokens)

      info["slots"][slot_name] = slot_value.strip()

    return info

In [ ]:
#loaded_model = tf.keras.models.load_model('model_epoch_18.keras', custom_objects={'JointIntentAndSlotFillingModel': JointIntentAndSlotFillingModel})

print(nlu("could you please change my reservation on december twenty first originally scheduled for twelve thirty in the morning to now be at four thirty five pm", tokenizer, joint_model, intent_names, slot_names))